# 面向对象高级编程 - 廖雪峰的官方网站

https://www.liaoxuefeng.com/wiki/1016959663602400/1017502538658208

# 使用 `__slots__`

In [8]:
class Student(object):
    pass

In [9]:
s = Student()
s.name = 'Michael' # 动态给实例绑定一个属性
print(s.name)

Michael


In [10]:
def set_age(self, age): # 定义一个函数作为实例方法
    self.age = age

给实例绑定一个方法

In [11]:
from types import MethodType
s.set_age = MethodType(set_age, s) # 给实例绑定一个方法
s.set_age(25) # 调用实例方法
s.age # 测试结果

25

对另一个实例是不起作用的：

In [14]:
s2 = Student() # 创建新的实例
s2.set_age(25) # 调用实例方法

AttributeError: 'Student' object has no attribute 'set_age'

为了给所有实例都绑定方法，可以给 `class` 绑定方法：

通常情况下，下面的`set_score`方法可以直接定义在`class`中，但动态绑定允许我们在程序运行的过程中动态给`class`加上功能，这在静态语言中很难实现。

In [15]:
def set_score(self, score):
    self.score = score

Student.set_score = set_score

In [16]:
s.set_score(100)
s.score

s2.set_score(99)
s2.score

100

99

## 使用__slots__

如果我们想要限制实例的属性怎么办？比如，只允许对 `Student` 实例添加 `name` 和 `age` 属性。

为了达到限制的目的， `Python` 允许在定义 `class` 的时候，定义一个特殊的 `__slots__` 变量，来限制该 `class` 实例能添加的属性：

In [1]:
class Student(object):
    __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称

In [2]:
s = Student() # 创建新的实例
s.name = 'Michael' # 绑定属性'name'
s.age = 25 # 绑定属性'age'
s.score = 99 # 绑定属性'score'

AttributeError: 'Student' object has no attribute 'score'

使用 `__slots__` 要注意， `__slots__` 定义的属性仅对当前类实例起作用，对继承的子类是不起作用的：`

In [4]:
class GraduateStudent(Student):
    pass

g = GraduateStudent()
g.score = 9999
g.score

9999

除非在子类中也定义 `__slots__`，这样，子类实例允许定义的属性就是自身的 `__slots__` 加上父类的 `__slots__` 

# 使用@property

在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把成绩随便改：

In [5]:
s = Student()
s.score = 9999

AttributeError: 'Student' object has no attribute 'score'

为了限制score的范围，可以通过一个 `set_score()` 方法来设置成绩，再通过一个`get_score()` 来获取成绩，这样，在 `set_score()` 方法里，就可以检查参数：

In [6]:
class Student(object):

    def get_score(self):
         return self._score

    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

In [7]:
s = Student()
s.set_score(60) # ok!
s.get_score()

s.set_score(9999)

60

ValueError: score must between 0 ~ 100!

还记得装饰器 `（decorator）` 可以给函数动态加上功能吗？

对于类的方法，装饰器一样起作用。 `Python` 内置的 `@property` 装饰器就是负责把一个方法变成属性调用的：

`@property` 的实现比较复杂，我们先考察如何使用。

把一个 `getter` 方法变成属性，只需要加上 `@property` 就可以了，此时， `@property` 本身又创建了另一个装饰器 `@score.setter` ，负责把一个 `setter` 方法变成属性赋值，于是，我们就拥有一个可控的属性操作：

In [8]:
class Student(object):

    @property
    def score(self):
        return self._score

    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

In [10]:
s = Student()
s.score = 60 # OK，实际转化为s.set_score(60)
s.score # OK，实际转化为s.get_score()

s.score = 9999

60

ValueError: score must between 0 ~ 100!

只定义 `getter` 方法，不定义 `setter` 方法就是一个 **只读属性** ：

`birth` 是可读写属性，而 `age` 就是一个只读属性，因为 `age` 可以根据 `birth` 和当前时间计算出来

In [17]:
class Student(object):

    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value

    @property
    def age(self):
        return 2015 - self._birth

## 练习

请利用 `@property` 给一个 `Screen` 对象加上 `width` 和 `height` 属性，以及一个只读属性 `resolution` 

In [15]:
class Screen():
    
    @property
    def width(self):
        return self._width
    
    @property
    def height(self):
        return self._height
    
    @property
    def resolution(self):
        return self._width * self._height
    

    @width.setter
    def width(self, value):
        self._width = value

    @height.setter
    def height(self, value):
        self._height = value
    



In [16]:
# 测试:
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

resolution = 786432
测试通过!


# 多重继承

In [18]:
class Animal(object):
    pass

# 大类:
# 哺乳动物
class Mammal(Animal):
    pass
# 鸟类
class Bird(Animal):
    pass

# 各种动物:
# 狗狗
class Dog(Mammal):
    pass
# 蝙蝠
class Bat(Mammal):
    pass
# 鹦鹉
class Parrot(Bird):
    pass
# 鸵鸟
class Ostrich(Bird):
    pass

现在，我们要给动物再加上 `Runnable` 和 `Flyable` 的功能，只需要先定义好 `Runnable` 和 `Flyable` 的类：

In [19]:
class Runnable(object):
    def run(self):
        print('Running...')

class Flyable(object):
    def fly(self):
        print('Flying...')

对于需要 `Runnable` 功能的动物，就多继承一个 `Runnable` ，例如 `Dog` ：

对于需要 `Flyable` 功能的动物，就多继承一个 `Flyable` ，例如 `Bat` ：

In [20]:
class Dog(Mammal, Runnable):
    pass

class Bat(Mammal, Flyable):
    pass

## MixIn

如果需要“混入”额外的功能，通过多重继承就可以实现，比如，让 `Ostrich` 除了继承自 `Bird` 外，再同时继承 `Runnable` 。这种设计通常称之为 `MixIn` 

为了更好地看出继承关系，我们把 `Runnable` 和 `Flyable` 改为 `RunnableMixIn` 和 `FlyableMixIn` 。

类似的，你还可以定义出肉食动物 `CarnivorousMixIn` 和植食动物 `HerbivoresMixIn` ，让某个动物同时拥有好几个 `MixIn` ：

```py
class Dog(Mammal, RunnableMixIn, CarnivorousMixIn):
    pass
```

 `MixIn` 的目的就是给一个类增加多个功能
 
 在设计类的时候，我们优先考虑通过多重继承来组合多个 `MixIn` 的功能，而不是设计多层次的复杂的继承关系。
 
 `Python` 自带的很多库也使用了 `MixIn` 。
 
 举个例子， `Python` 自带了 `TCPServer` 和 `UDPServer` 这两类网络服务，而要同时服务多个用户就必须使用 **多进程** 或 **多线程模型** ，这两种模型由 `ForkingMixIn` 和 `ThreadingMixIn` 提供。通过组合，我们就可以创造出合适的服务来。

比如，编写一个 **多进程模式的TCP** 服务，定义如下：

```py
class MyTCPServer(TCPServer, ForkingMixIn):
    pass
```

编写一个 **多线程模式的UDP**m服务，定义如下：

```py
class MyUDPServer(UDPServer, ThreadingMixIn):
    pass
```


# 定制类

## `__str__`

我们先定义一个 `Student` 类，打印一个实例：

In [21]:
class Student(object):
    def __init__(self, name):
        self.name = name
        
print(Student('Michael'))

`__str__()` 方法

In [24]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name: %s)' % self.name
    
print(Student('Michael'))

s = Student('Michael')
s

Student object (name: Michael)


`__str__()`  `__repr__()` ，两者的区别是 `__str__()` 返回用户看到的字符串，而 `__repr__()` 返回程序开发者看到的字符串，也就是说， `__repr__()` 是为调试服务的

## `__repr__()` 方法

In [25]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Student object (name: %s)' % self.name
    
print(Student('Michael'))

s = Student('Michael')
s

Student object (name: Michael)


Student object (name: Michael)

## `__iter__()`方法

`Python` 的 `for` 循环就会不断调用该迭代对象的 `__next__()` 方法拿到循环的下一个值，直到遇到 `StopIteration` 错误时退出循环。

In [31]:
class Fib(object):
    
    def __init__(self, stop=10):
        self.a, self.b = 0, 1 # 初始化两个计数器a，b
        self.stop = stop
    def __iter__(self):
        return self # 实例本身就是迭代对象，故返回自己

    def __next__(self):
        self.a, self.b = self.b, self.a + self.b # 计算下一个值
        if self.b > self.stop: # 退出循环的条件
            raise StopIteration()
        return self.a # 返回下一个值

In [32]:
for n in Fib():
    print(n)

1
1
2
3
5


## `__getitem__`

要表现得像 `list` 那样按照下标取出元素，需要实现 `__getitem__()` 方法：

In [34]:
Fib()[5] # __iter__()方法 *不行*

TypeError: 'Fib' object does not support indexing

In [35]:
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b
        return a

In [37]:
f = Fib()

f[0]

f[10]

1

89

切片方法

`__getitem__()` 传入的参数可能是一个 `int` ，也可能是一个切片对象 `slice` ，所以要做判断

In [39]:
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int): # n是索引
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice): # n是切片
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
            return L

In [40]:
f = Fib()

f[0:5]

f[:10]

[1, 1, 2, 3, 5]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

如果把对象看成 `dict` ， `__getitem__()` 的参数也可能是一个可以作 `key` 的 `object` ，例如 `str` 。

与之对应的是 `__setitem__()` 方法，把对象视作 `list` 或 `dict` 来对集合赋值。

最后，还有一个 `__delitem__()` 方法，用于删除某个元素。

## `__getattr__` 

In [45]:
class Student(object):
    
    def __init__(self):
        self.name = 'Michael'

In [46]:
s=Student()

s.name
s.score

'Michael'

AttributeError: 'Student' object has no attribute 'score'

In [59]:
class Student(object):

    def __init__(self):
        self.name = 'Michael'

    def __getattr__(self, attr):
        if attr=='score':
            return 99

In [48]:
s=Student()

s.name
s.score

'Michael'

99

注意，只有在没有找到属性的情况下，才调用 `__getattr__` ，已有的属性，比如 `name` ，不会在 `__getattr__` 中查找。

要让 `class` 只响应特定的几个属性，我们就要按照约定，抛出 `AttributeError` 的错误：

In [62]:
class Student2(Student):

    def __init__(self):
        super().__init__()
    
    def __getattr__(self, attr):
        if attr=='age':
            return lambda: 25
        raise AttributeError('\'Student\' object has no attribute \'%s\'' % attr)

In [64]:
s=Student2()

s.name 
s.age() # 返回lambda 函數
s.score

'Michael'

25

AttributeError: 'Student' object has no attribute 'score'

现在很多网站都搞 `REST API` ，比如新浪微博、豆瓣啥的，调用 `API` 的 `URL` 类似：

```
http://api.server/user/friends
http://api.server/user/timeline/list
```
如果要写 `SDK` ，给每个 `URL` 对应的 `API` 都写一个方法，那得累死，而且， `API` 一旦改动， `SDK` 也要改。

利用完全动态的 `__getattr__` ，我们可以写出一个链式调用：

In [67]:
class Chain(object):

    def __init__(self, path='Strat'):
        self._path = path

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__

In [68]:
Chain().status.user.timeline.list

Strat/status/user/timeline/list

还有些 `REST API` 会把参数放到 `URL` 中，比如 `GitHub` 的 `API` ：

```
GET /users/:user/repos
```

调用时，需要把 `:user` 替换为实际用户名

In [89]:
class Chain(object):

    def __init__(self, path='Strat'):
        self._path = path
        
    def users(self, s):
        return Chain(f'{self._path}/{s}')
    
    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__

In [90]:
Chain().users('michael').repos

Strat/michael/repos

## `__call__`

任何类，只需要定义一个 `__call__()` 方法，就可以直接对实例进行调用

In [98]:
class Student(object):
    def __init__(self, name):
        self.name = name
        
s = Student('Michael')
callable(s)
s()


False

TypeError: 'Student' object is not callable

In [100]:
class Student(object):
    def __init__(self, name):
        self.name = name

    def __call__(self):
        print('My name is %s.' % self.name)

In [101]:
s = Student('Michael')
callable(s)
s() # self参数不要传入

True

My name is Michael.
